# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [2]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  # df/da = -3* a**2 - 0.5 * (a**(-0.5))
  # df/db = 3*cos(3b) + 2.5 * (b ** 1.5)
  # df/dc = -1/(c**2) * -1
  return [-3*a**2 - 0.5 * (a**(-0.5)), 3*cos(3*b) + (2.5 * (b**1.5)), 1/(c**2)]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [3]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

h = 0.0000001
a,b,c = 2, 3, 4
df_da = (f(a+h, b, c) - f(a, b, c)) / h
df_db = (f(a,b+h,c) - f(a,b,c)) / h
df_dc = (f(a, b, c+h) - f(a, b, c)) / h

# -----------
numerical_grad = [df_da, df_db, df_dc] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553966448771
OK for dim 1: expected 10.25699027111255, yours returns 10.256990385926201
OK for dim 2: expected 0.0625, yours returns 0.06249999628948899


In [4]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

h = 0.0000001
a,b,c = 2, 3, 4
df_da = ((f(a+h, b, c) - f(a-h, b, c)) / (2*h))
df_db = ((f(a,b+h,c) - f(a,b-h,c)) / (2*h))
df_dc = ((f(a, b, c+h) - f(a, b, c-h)) / (2*h))

# -----------
numerical_grad2 = [df_da, df_db, df_dc] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553380251014
OK for dim 1: expected 10.25699027111255, yours returns 10.256990252699438
OK for dim 2: expected 0.0625, yours returns 0.06250000073038109


## section 2: support for softmax

In [67]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    if isinstance(other, int) or isinstance(other, float): other = Value(other)
    out = self.data * other.data
    out_val = Value(data=out, _children=(self, other), _op='*', label="*")

    def _backward():
      c1, c2 = out_val._prev
      c1.grad += out_val.grad * c2.data
      c2.grad += out_val.grad * c1.data

    out_val._backward = _backward

    return out_val

  def __neg__(self):
    return self * -1

  def __sub__(self, other):
    return self + -other

  def __radd__(self, other):
    return self + other

  def __rmul__(self, other):
    return self * other

  def __truediv__(self, other):
    return self * other**-1

  # exp
  def exp(self):
    data = exp(self.data)
    out = Value(data=data, _children=(self, ), _op='exp', label='exp')

    def _backward():
      self.grad += out.data * out.grad

    out._backward = _backward

    return out

  # pow
  def __pow__(self, other):
    assert isinstance(other, (float, int))
    out = Value(data=self.data ** other, _children=(self,), _op=f"**{other}")

    def _backward():
      self.grad += (other * (self.data ** (other - 1))) * out.grad

    out._backward = _backward

    return out


  # log
  def log(self):
    out = Value(data=log(self.data), _children=(self,), _op="log", label="log")

    def _backward():
      self.grad += (1/self.data) * out.grad

    out._backward = _backward

    return out

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [68]:
a = Value(2.71)
b = Value(3.0)
c = a.log()
c

Value(data=0.9969486348916096)

In [58]:
c.backward()

In [59]:
c.grad, b.grad, a.grad

(1.0, 0.0, 0.36900369003690037)

In [69]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [143]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
from torch.nn import Softmax

logits_tensor = torch.Tensor([0.0, 3.0, -2.0, 1.0]).double()
logits_tensor.requires_grad = True
p = Softmax()
probs = p(logits_tensor)
loss = probs[3].log()*-1
loss.backward()
logits_tensor_grad = logits_tensor.grad

for dim in range(4):
  ok = 'OK' if abs(logits_tensor_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits_tensor_grad[dim]}")



OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535046
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625321
OK for dim 2: expected 0.005653302662216329, yours returns 0.00565330266221633
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400987
